You can generate crystal structures for a given chemical composition partly using CrySPY  
(Here no CSP, only structure generation)  

See the document of CrySPY for details of arguments  
https://tomoki-yamashita.github.io/CrySPY/input_file.html

# Import

In [ ]:
from __future__ import print_function

In [ ]:
# ---------- set the path of your CrySPY
#            check /your_cryspy_path/CrySPY/gen_struc
import sys
sys.path.append('/Users/yamashita06/Dropbox/research/program/CSP/CrySPY/CrySPY_git/CrySPY')

In [ ]:
import gen_struc as gs

In [ ]:
# ---------- for data preparetin in EA
import pickle

# Random structure generation

## Rnd_struc_gen class  
Rnd_struc_gen(natot, atype, nat, minlen, maxlen, dangle, mindist, maxcnt=200, symprec=0.001)  

There are two ways  
  1) with space group infromation (symmetry)  
  2) without space gruop information

In [ ]:
# ---------- example of Si4O8
natot = 12
atype = ['Si', 'O']
nat = [4, 8]
minlen = 4
maxlen = 10
dangle = 20.0
mindist = [[1.8, 1.3],
           [1.3, 1.5]]

In [ ]:
# ---------- instantiate Rnd_struc_gen class
rsg = gs.random.Rnd_struc_gen(natot, atype, nat, minlen, maxlen, dangle, mindist)

## When you use space group information (symmetry) for structure generation

find_wy program is required  
tmp_gen_struc directory is temporaryly made in the current directory

In [ ]:
# ---------- setting 
nstruc = 5                    # number of structures
spgnum = 'all'                # all space gruop (default: all)
# spgnum = [100, 150]         # if you want to use only space group 100 and 150
id_offset = 0                 # Structure ID starts from id_offset (default: 0)
#init_pos_path = None         # if you specify a file path,
init_pos_path = './POSCARS'   #    structure data in POSCAR format are written in the file
                              #    you can open this file using VESTA

In [ ]:
# ---------- path of find_wy
fwpath = '/Users/yamashita06/local/src/find_wy/find_wy'

In [ ]:
# ---------- generate structure
rsg.gen_with_spg(nstruc, spgnum, id_offset, init_pos_path, fwpath=fwpath)

You can access generated structure data to see rsg.init_struc_data  
rsg.init_sturc_data is dict type like {0: struc_data_0, 1: struc_data_1, ... , ID: struc_data_ID}

In [ ]:
# ---------- e.g. ID 0
rsg.init_struc_data[0]

## When you DO NOT use space group information (symmetry) for structure generation

find_wy program is NOT required  

In [ ]:
# ---------- setting 
nstruc = 5                    # number of structures you will generate
spgnum = 0                    # no symmetry
id_offset = 0                 # Structure ID starts from id_offset (default: 0)
#init_pos_path = None         # if you specify a file path,
init_pos_path = './POSCARS'   #    structure data in POSCAR format are written in the file
                              #    you can open this file using VESTA

In [ ]:
# ---------- generate structure
rsg.gen_wo_spg(nstruc, id_offset, init_pos_path)

You can access generated structure data to see rsg.init_struc_data  
rsg.init_sturc_data is dict type like {0: struc_data_0, 1: struc_data_1, ... , ID: struc_data_ID}

In [ ]:
# ---------- e.g. ID 3
rsg.init_struc_data[3]

# Structure generation by EA
Here, let us use sample data of Si16 as parent data

Si16, 10 structures

## Load parent data

In [ ]:
def load_pkl(filename):
    with open(filename, 'rb') as f:
        pkl_data = pickle.load(f)
    return pkl_data

In [ ]:
# ---------- load optimized structure data
opt_struc_data = load_pkl('./sample_data_Si16/opt_struc_data.pkl')

In [ ]:
# ---------- load rslt_data for energy datta of parent structure
rslt_data = load_pkl('./sample_data_Si16/rslt_data.pkl')
rslt_data    # pandas DataFrame

In [ ]:
# ---------- get energy data as dict: key is structure ID
fitness = dict(zip(rslt_data['Struc_ID'].values, rslt_data['Energy'].values))

In [ ]:
fitness

If you fail to load the "rslt_data.pkle" file owing to difference version of pandas or something,  
use the following cell

In [ ]:
# fitness = {
#  0: -67.302877312801,
#  1: -64.98995856317622,
#  2: -62.044391121520356,
#  3: -64.9899614176506,
#  4: -62.05936739490061,
#  5: -61.439897269638045,
#  6: -63.593619764605805,
#  7: -63.90257519925848,
#  8: -65.12751185011778,
#  9: -62.35235718827326}

## Select parents
There are two ways:  
  1) Tournament selection  
  2) Roulette selection  

Here let us try tournament selection

In [ ]:
# ---------- setting
elite_struc = None     # You can add additional parents,
elite_fitness = None   #     if you add elite_struc and elite fitness
fit_reverse = False    # False means minimum search
n_fittest = 5          # number of survival of the fittest

In [ ]:
# ---------- instantiate Select_parents class
sp = gs.EA.Select_parents(opt_struc_data, fitness, elite_struc, elite_fitness,
                          fit_reverse, n_fittest)

In this setting stable 5 (=n_fittest) structures can become candidates to be selected  
You can check survivors

In [ ]:
# ---------- top 5 structures
sp.ranking_dedupe

In [ ]:
# ---------- setting for tournament selection
t_size = 2
sp.set_tournament(t_size)

## Generate offspring

In [ ]:
# ---------- setting
symprec = 0.001  # precision for symmetry (default: 0.001)
id_start = 10    # next structure ID (here we already have up to 9)
init_pos_path = './POSCARS'

In [ ]:
# ---------- instantiate EA_generation class
eagen = gs.EA.EA_generation(sp, symprec, id_start, init_pos_path)

In [ ]:
# --------- setting for crossover and strain
atype = ['Si']
nat = [16]
mindist = [[1.5]]
maxcnt_ea = 100    # default value

## Crossover
Let us make 5 structures by crosover operation

In [ ]:
# ---------- setting for crossover
crs_lat = 'equal'    # 'eaual' or 'random'
crs_func = 'OP'      # one point crossover or two point crossover (TP)
nat_diff_tole = 4    # default

In [ ]:
# ---------- instantiate Crossover class
co = gs.EA.Crossover(atype, nat, mindist, crs_lat, crs_func,
               nat_diff_tole, maxcnt_ea)

In [ ]:
# ---------- generate
eagen.gen_crossover(n_crsov=5, co=co)

You can see generated structue data in eagen.offspring (dict)

In [ ]:
# ---------- e.g. ID 13
eagen.offspring[13]

## Permutation
we skip permutaion because of one element system

In [ ]:
# # ---------- setting
# n_times = 1    # default

In [ ]:
# # ---------- instantiate Permutation class
# pm = gs.EA.Permutation(atype, mindist, ntimes, maxcnt_ea)

In [ ]:
#eagen.gen_permutation(nperm=4, pm=pm)

## Strain
Let us make 5 structures by strain

In [ ]:
# ---------- setting for strain
sigma_st = 0.5    # Standard deviation for strain

In [ ]:
# ---------- instantiate Strain class
st = gs.EA.Strain(atype, mindist, sigma_st, maxcnt_ea)

In [ ]:
# ---------- generate
eagen.gen_strain(n_strain=5, st=st)

You can see generated structue data in eagen.offspring (dict)